In [3]:
import pyodbc as db 
import pandas as pd 

conn = db.connect('DRIVER={SQL Server};'
                      'SERVER=10.168.2.168;'
                      'DATABASE=TCPL_SECONDARY;'
                      'UID=sa;'
                      'PWD=erp;')
sku_df = pd.read_sql_query(""" 
                DECLARE @date date = GETDATE(); 
                declare @current_day int = right(convert(varchar(8),getdate(), 112), 2)
                
                select srItnameTarget.srname,srItnameTarget.Item, srItnameTarget.skuname,
                [Months Target(Tk)], 
                [MTD Target(Tk)], 
                isnull([MTD Sales(Tk)], 0) as [MTD Sales(Tk)],
                --(isnull([Months Sales(Tk)], 0)/[MTD Target(Tk)]) as 'achiv %',
                [Months Target(Kg)],
                [MTD Target(Kg)], 
                [MTD Sales(Kg)]
                 from
                (select targetsr.srid,srname,sku.skuid as Item, sku.ShortName as skuname,sum(TargetValue) as [Months Target(Tk)],
                cast((sum(TargetValue) /  cast(DAY(EOMONTH ( @date )) as int)) * @current_day as int) as [MTD Target(Tk)],
                sum(TargetQty*Weight)/1000 as [Months Target(Kg)], 
                cast((((sum(TargetQty*Weight)/1000)/ cast(DAY(EOMONTH ( @date )) as int)) * @current_day) as int) as [MTD Target(Kg)]
                 from 
                (Select * from TargetDistributionItemBySR where yearmonth = convert(varchar(6),DATEADD(D,0,GETDATE()),112) and srid=22) as targetsr
                left join 
                (select * from Hierarchy_Emp) as emp
                on targetsr.srid=emp.SRID
                left join 
                (select * from Hierarchy_SKU) as Sku
                on sku.SKUID=targetsr.SKUID
                group by srname,sku.skuid,shortname, targetsr.srid
                ) as srItnameTarget
                right join 
                (select salesitem.SRID,srname,sku.skuid as Item,isnull(sum((Quantity-FreeQty)*InvoicePrice), 0) as [MTD Sales(Tk)],
                isnull(sum((Quantity-FreeQty)*Weight)/1000, 0) as [MTD Sales(Kg)] from 
                (selecT Item.*,Srid from 
                (select * from SalesInvoices where InvoiceDate between convert(varchar(10),DATEADD(mm, DATEDIFF(mm, 0, GETDATE()), 0),126)
                and convert(varchar(10),DATEADD(D,0,GETDATE()),126) and srid=22) as Sale
                inner join 
                (select * from SalesInvoiceItem) as Item
                on sale.InvoiceID=item.InvoiceID) as salesitem
                left join 
                (select * from Hierarchy_Emp) as emp
                on salesitem.srid=emp.SRID
                left join 
                (select * from Hierarchy_SKU) as Sku
                on sku.SKUID=salesitem.SKUID
                group by srname,sku.skuid,salesitem.SRID) as srItnamesale
                on srItnameTarget.srid=srItnamesale.srid
                and srItnameTarget.Item=srItnamesale.Item """, conn)


In [4]:
sku_df.columns

Index(['srname', 'Item', 'skuname', 'Months Target(Tk)', 'MTD Target(Tk)',
       'MTD Sales(Tk)', 'Months Target(Kg)', 'MTD Target(Kg)',
       'MTD Sales(Kg)'],
      dtype='object')

In [25]:
sku_df.to_excel('./Data/sku_wise_target_sales.xlsx', index=False)